# Interfaces: From Protocols to ABCs

ABC: Abstract Base Class

Python2.6 才引入抽象基类。

> 抽象基类与描述符、元类一样，是构建工具框架的工具。因此还有少数Python开发者编写的抽象基类不会对用户施加不必要的限制。

## 定义抽象基类的子类

In [2]:
import collections

In [5]:
# help(collections.abc.MutableSequence)

In [6]:
collections.abc.MutableSequence.__mro__

(collections.abc.MutableSequence,
 collections.abc.Sequence,
 collections.abc.Reversible,
 collections.abc.Collection,
 collections.abc.Sized,
 collections.abc.Iterable,
 collections.abc.Container,
 object)

```
class MutableSequence(Sequence)
 |  All the operations on a read-only sequence.
 |  
 |  Concrete subclasses must override __new__ or __init__,
 |  __getitem__, and __len__.
 |  
 |  Method resolution order:
 |      MutableSequence
 |      Sequence
 |      Reversible
 |      Collection
 |      Sized
 |      Iterable
 |      Container
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __delitem__(self, index)
 |  
 |  __iadd__(self, values)
 |  
 |  __setitem__(self, index, value)
 |  
 |  append(self, value)
 |      S.append(value) -- append value to the end of the sequence
 |  
 |  clear(self)
 |      S.clear() -> None -- remove all items from S
 |  
 |  extend(self, values)
 |      S.extend(iterable) -- extend sequence by appending elements from the iterable
 |  
 |  insert(self, index, value)
 |      S.insert(index, value) -- insert value before index
 |  
 |  pop(self, index=-1)
 |      S.pop([index]) -> item -- remove and return item at index (default last).
 |      Raise IndexError if list is empty or index is out of range.
 |  
 |  remove(self, value)
 |      S.remove(value) -- remove first occurrence of value.
 |      Raise ValueError if the value is not present.
 |  
 |  reverse(self)
 |      S.reverse() -- reverse *IN PLACE*
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset({'__delitem__', '__getitem__', '__len_...
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Sequence:
 |  
 |  __contains__(self, value)
 |  
 |  __getitem__(self, index)
 |  
 |  __iter__(self)
 |  
 |  __reversed__(self)
 |  
 |  count(self, value)
 |      S.count(value) -> integer -- return number of occurrences of value
 |  
 |  index(self, value, start=0, stop=None)
 |      S.index(value, [start, [stop]]) -> integer -- return first index of value.
 |      Raises ValueError if the value is not present.
 |      
 |      Supporting start and stop arguments is optional, but
 |      recommended.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods inherited from Reversible:
 |  
 |  __subclasshook__(C) from abc.ABCMeta
 |      Abstract classes can override this to customize issubclass().
 |      
 |      This is invoked early on by abc.ABCMeta.__subclasscheck__().
 |      It should return True, False or NotImplemented.  If it returns
 |      NotImplemented, the normal algorithm is used.  Otherwise, it
 |      overrides the normal algorithm (and the outcome is cached).
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Sized:
 |  
 |  __len__(self)
 ```

In [10]:
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamons clubs hearts'.split
    
    def __init__(self):
        self._cards = [ Card(rank, suit) for suit in suits for rank in ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def insert(self, position, val):
        self._cards.insert(position, val)

import时，Python不会检查抽象方法的实现，在运行时实例化对象时才会真正的检查。因此，如果没有正确实现某个抽象方法，Python不会抛出TypeError异常。

![](https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=4215643129,2776770392&fm=15&gp=0.jpg)

```
要想实现子类，可以覆盖从抽象基类中继承的方法，一更高效的方式重新实现。例如，__contains__方法会全面扫描序列，可是，如果你定义的序列安顺序保存元素，那就可以重新定义__contains__方法，使用bisect函数做二分查找，从而提高搜索速度。
```

## 标准库中的抽象基类

从Python2.6开始，标准库中提供了抽象基类。大多数抽象基类在`collections.abc`模块中定义，不过其他地方也有。例如：numbers和io包中有一些抽象基类。

### collections.abc模块中的抽象基类

![](https://bugs.python.org/file47357/base.png)

Iterable，Container和sized

```
各个集合应该继承这三个抽象基类，或者至少实现兼容的协议。Iterable通过__iter__方法支持迭代，Container通过__contains__方法支持in运算符，Sized通过__len__方法支持len()函数。
```

### 抽象基类的数字塔

numbers包定义的“数字塔”。

* Number
* Complex
* Real
* Rational
* Integral

In [11]:
import numbers

In [14]:
numbers.Integral.__mro__

(numbers.Integral,
 numbers.Rational,
 numbers.Real,
 numbers.Complex,
 numbers.Number,
 object)

> decimal.Decimal没有注册为numbers.Real的虚拟子类。没注册的原因是，如果你的程序要Decimal的精度，要防止与其他低精度的数字类型混淆，尤其是浮点数。

## 定义并使用一个抽象基类

Tombola抽象基类命名为Tombola，有四个方法，其中两个是抽象方法。

* .load(...): put items to the container.
* .pick(...): remove one item at rondom from the container, returning it.


The concrete methods are:

* .loaded(): return True if there is at least one in the container
* .inspect(): return a sorted tuple built from the items currently in the container, without chaning ite contents.

In [16]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable"""

    @abc.abstractmethod
    def pick(self):
        """
        Remove item at random, returning it.
        This method should riase `LookupError` when the instance is empty.
        """
        
    def loaded(self):
        """Return True if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())
    
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError():
                break
                
        self.load(items)
        return tuple(sorted(items))

> An abstract method can actually have an implementation. Even if it does, subclass will still be forced to override it, but they will be able to invoke the abstract method with super(), adding functionality to it instead of implementing from scratch.

### 定义Tombola ABC的子类

In [18]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try: 
            self._items.pop()
        except IndexErrore:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

### Tombola 虚拟子类

A Virtual Subclass of Tombola.

注册虚拟子类的方式在抽象基类上调用register方法。注册的类会变成抽象基类的虚拟子类，而且issubclass和isinstance等函数都能识别，但是注册的类不会从抽象基类中继承任何方法和属性。

This is done by calling a register method on the ABC. The registered class then becomes a virtual subclass of the ABC, and will recoginized as such by functions like issublcass and isinstance, but it will not inherit any methods or attributes from the ABC.

> 虚拟子类不会继承注册的ABC，而且任何时候都不会检查它是否符合ABC的接口，即便在实例化时也不会检查。为了遍运行时错误，虚拟子类要实现所需的全部方法。

> Virtual subclass do not inherit from their registered ABCs, and not checked for conformance to the ABC interface  at any time, not even when they are instantiated. It's up to the subclass to actually implement all the methods needed to avoid runtime errors.

Register方法通常作为普通的函数调用，不过也可以作为装饰器使用。我们使用装饰器语法实现了TomboList类，这是Tombola的一个虚拟子类。

![](https://hotttao.gitbooks.io/fluent-python-note/image/abstract_register.png)

In [24]:
from random import randrange


@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self: # 从list中即成了__bool__方法，列表为空时返回True
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
            
    load = list.extend
    
    def loaded():
        return bool(self)
    
    def inpsect(self):
        return tuple(sorted(self))
            

In [25]:
issubclass(TomboList, Tombola)

True

In [26]:
isinstance(TomboList(range(10)), Tombola)

True

In [27]:
TomboList.__mro__

(__main__.TomboList, list, object)

```
TomboList.__mro__中没有Tombola，因此Tombolist没有从Tombola中即成任何方法。
```

## Tombola子类的测试方法

How the TOmbola Subclass Were Tested

```
__subclasses__()

    这个方法类的直接子类列表，不含虚拟子类。
```

In [28]:
Tombola.__subclasses__()

[__main__.BingoCage]

## 延伸阅读

**Python是弱类型语言吗**

Python 强类型、动态语言。

**强类型和弱类型**

如果一门语言很少隐式转换类型，说明他是强类型语言；如果经常这么做，说明他是弱类型语言。Java、C++和Python是强类型语言。PHP、JavaScrpt和Perl是弱类型语言。

**静态类型和动态类型**

在编译时检查类型的语言是静态类型语言，在运行时检查类型的语言是动态类型语言。静态类型语言需要声明类型。

## References

* *Fluent Python, Interfaces From Protocols to ABCs*
* [Python PEP 3119 -- Introducting Abstract Classes](https://www.python.org/dev/peps/pep-3119/)